In [2]:
! pip install allensdk

     |████████████████████████████████| 1.6 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 283 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 26.5 MB 142 kB/s  eta 0:00:01
     |████████████████████████████████| 668 kB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 39.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 42.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 53.3 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 11.6 MB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 6.3 MB/s  eta 0:00:01
     |█████████████████████████████

  Created wheel for glymur: filename=Glymur-0.8.19-py3-none-any.whl size=2721998 sha256=f2bf5d8658a5701d2b7cd482383cbfe839f1380854da31ab248b467fec1a0594
  Stored in directory: /home/jovyan/.cache/pip/wheels/36/bc/bd/1786279b44db4cbd6ded18090d02978fc31e78e3e516551bb4
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=b7ef7c755b081c01589b74c325ded19a20375c964a4649fb8ad8d4d878b4fa8f
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built glymur future


In [3]:

from allensdk.api.queries.rma_api import RmaApi
from allensdk.api.cache import Cache
from allensdk.api.queries.grid_data_api  import GridDataApi

import numpy as np
import pandas as pd

In [4]:
# we use the RmaApi to query specific information, such as the section data sets of a specific gene
# for docs, see: https://alleninstitute.github.io/AllenSDK/allensdk.api.queries.rma_api.html
rma = RmaApi() 

# there might be a way to retrieve data in higher resolution, as stated here (default is 25, 10 is also available - but resolution is ignored for download_gene_expression_grid_data)
# https://alleninstitute.github.io/AllenSDK/_modules/allensdk/api/queries/grid_data_api.html
# See `Downloading 3-D Projection Grid Data <http://help.brain-map.org/display/api/Downloading+3-D+Expression+Grid+Data#name="Downloading3-DExpressionGridData-DOWNLOADING3DPROJECTIONGRIDDATA">`_
gdApi = GridDataApi()

# the cache_writeer allows us to easily cache the results
cache_writer = Cache()        

geneAcronym = "Gabra4"
# http://api.brain-map.org/examples/rma_builder/index.html
# http://api.brain-map.org/examples/rma_builder/rma_builder.html
# https://allensdk.readthedocs.io/en/latest/data_api_client.html
sectionDataSets = pd.DataFrame( # wrap is told to be deprecated, but there is no information on what to use instead :(
    cache_writer.wrap(rma.model_query,
                        path='cache\\section-data-sets.json',
                        cache=True, # the semantics of this function are a bit weird. providing True means: add it to the cache
                        model='SectionDataSet',
                        filters={'failed':'false'},
                        include=f"genes[acronym$il{geneAcronym}],products[id$eq1]", # $il = case-insensitive like | yes, weird notation... id = 1 = mouse brain atlas (not developing!)
                        num_rows='all'))
# model's documentation: http://api.brain-map.org/doc/SectionDataSet.html
# https://community.brain-map.org/t/attempting-to-download-substructures-for-coronal-p56-mouse-atlas/174/2

experiments = []
#zipfile.ZipFile(request.urlretrieve(self.url)[0]).read("gridAnnotation.raw")))
# http://help.brain-map.org/display/mousebrain/Documentation
# TODO: i think this is wrong. we have different types of grid data: saggital and coronal. are the annotations in the same order for both??
annotationsU = np.fromfile("annotations/P56_Mouse_gridAnnotation/gridAnnotation.raw", dtype="uint32")
annotations = np.fromfile("annotations/P56_Mouse_gridAnnotation/gridAnnotation.raw", dtype="int32")

# for Mouse P56, structure_graph_id = 1 according to http://help.brain-map.org/display/api/Atlas+Drawings+and+Ontologies
# structure_map = StructureMap(reference_space_key = 'annotation/ccf_2017', resolution=25).get(structure_graph_id=1)

for index, row in sectionDataSets.iterrows(): # https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
    exp_id = row['id']
    exp_path = f"data/{exp_id}/"

    #refSp = ReferenceSpaceApi()
    #anns = refSp.download_mouse_atlas_volume(age=15, volume_type=GridDataApi.ENERGY, file_name=f'cache\\mouse_atlas_volume.zip')
    #print(anns)

    # http://help.brain-map.org/display/mousebrain/API

    try:
        gdApi.download_gene_expression_grid_data(exp_id, GridDataApi.ENERGY, exp_path)

        expression_levels = np.fromfile(exp_path + "energy.raw",  dtype=np.float32)

        # According to the docs here: http://help.brain-map.org/display/api/Downloading+3-D+Expression+Grid+Data
        # we have "A raw uncompressed float (32-bit) little-endian volume representing average expression energy per voxel. A value of "-1" represents no data. This file is returned by default if the volumes parameter is null."
        # energy = numpy.array(list(struct.iter_unpack("<f", open(exp_path + "energy.raw", "rb").read()))).flatten() # way too complicated, but there is a delta in mean and sum. what is the right value??
        data = pd.DataFrame({"expression_level": expression_levels, "structure_id": annotations})

        # TODO: there is something wrong. some expression_levels are assigned to a structure of id 0. same is true for Jure's approach
        data = data[(data.expression_level != -1)] # (data.structure_id != 0) & ]

        
        print(data[(data.structure_id==0) & (data.expression_level>0)])

    except Exception as e:
        print(f"Error retrieving experiment {exp_id}: {str(e)}")


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Function wrap is deprecated. 
2020-11-08 11:22:00,688 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download/75551483?include=energy
2020-11-08 11:22:01,255 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download/71924402?include=energy


       expression_level  structure_id
22284      2.352117e-02             0
22285      1.878159e-02             0
22286      9.601472e-03             0
22335      1.081399e-09             0
22336      8.580686e-08             0
...                 ...           ...
82405      3.652525e-03             0
82406      3.319370e-03             0
82407      3.915340e-03             0
82408      2.722567e-03             0
82409      8.182127e-04             0

[14780 rows x 2 columns]
        expression_level  structure_id
840         1.853983e-09             0
842         8.163044e-07             0
843         4.060922e-06             0
844         1.709051e-06             0
845         2.794895e-07             0
...                  ...           ...
158563      6.785410e-08             0
158564      3.602006e-07             0
158565      9.219777e-07             0
158566      5.619149e-07             0
158567      9.208617e-07             0

[29905 rows x 2 columns]
